In [20]:
import os
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\i2cat\Anaconda

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-1.21.0               |             py_0          58 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          92 KB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-1.21.0-py_0




geographiclib-1.50   | 34 KB     |            |   0% 
geographiclib-1.50   | 34 KB     | ####7      |  47% 
geographiclib-1.50   | 34 KB     | ########## | 100% 

geopy-1.21.0         | 58 KB     |            |   0% 
geopy-1.21.0         | 58 KB

usage: conda-script.py [-h] [-V] command ...
conda-script.py: error: unrecognized arguments: # uncomment this line if you haven't completed the Foursquare API lab


In [21]:
#Definició de dataframe de veïnes
col_names= ['Codi Veina','Nom','Barri']
df_veines= pd.DataFrame(columns=col_names)
df_veines.head()

,Codi Veina,Nom,Barri


In [22]:
#Afegir veïns a la base de dades
#cod_vei= #camp de input del codi client
#nom_vei= #camp de input del nom client
#barri= #nom del barri (estaria guay que fos una lista i que el vei seleciona, per evitar problemes de ortografia)
#df_veines.loc[len(df_veines)] = [cod_vei, nom_vei, barri]
#df_veines.head()

In [23]:
df_barris=pd.read_csv(r'C:\Users\i2cat\Dades Obertes\Hackovid\Neighborhoods_Barcelona.csv')
print(df_barris.shape)
df_barris.head()

(11396, 7)


,Any,Codi_Districte,Nom_Districte,Codi_Barri,Nom_Barri,Nacionalitat,Nombre
0,2015,1,Ciutat Vella,1,el Raval,Espanya,1097
1,2015,1,Ciutat Vella,2,el Barri Gòtic,Espanya,521
2,2015,1,Ciutat Vella,3,la Barceloneta,Espanya,457
3,2015,1,Ciutat Vella,4,"Sant Pere, Santa Caterina i la Ribera",Espanya,752
4,2015,2,Eixample,5,el Fort Pienc,Espanya,717


In [24]:
df_barris.drop(['Any','Nacionalitat','Nombre','Codi_Districte'],axis=1,inplace=True)
df_barris.head()

,Nom_Districte,Codi_Barri,Nom_Barri
0,Ciutat Vella,1,el Raval
1,Ciutat Vella,2,el Barri Gòtic
2,Ciutat Vella,3,la Barceloneta
3,Ciutat Vella,4,"Sant Pere, Santa Caterina i la Ribera"
4,Eixample,5,el Fort Pienc


In [25]:
df_barris.columns=['Districte','Codi de Barri','Barri']
print(df_barris.shape)
df_barris.head()

(11396, 3)


,Districte,Codi de Barri,Barri
0,Ciutat Vella,1,el Raval
1,Ciutat Vella,2,el Barri Gòtic
2,Ciutat Vella,3,la Barceloneta
3,Ciutat Vella,4,"Sant Pere, Santa Caterina i la Ribera"
4,Eixample,5,el Fort Pienc


In [26]:
df_barris.drop(df_barris.index[73:11395],inplace=True, axis=0)
print(df_barris.shape)

(74, 3)


In [27]:
latitude1 = 41.3887901
longitude1 = 2.1589899
print('The geograpical coordinate of Barcelona are {}, {}.'.format(latitude1, longitude1))

The geograpical coordinate of Barcelona are 41.3887901, 2.1589899.


In [29]:
#get coordinates for the neighborhoods of Barcelona
neigh_bar_coord=[]
for name in df_barris['Barri']:
    address_neig = ('{},Barcelona, BCN'.format(name))
    geolocator = Nominatim(user_agent="h")
    location = geolocator.geocode(address_neig, timeout=10)
    if location == None:
        latitude=0
        longitude=0
    else :
        latitude = location.latitude
        longitude = location.longitude
    neigh_bar_coord.append([name,latitude,longitude])

In [32]:
neig_coord_df = pd.DataFrame(neigh_bar_coord)
neig_coord_df.columns = ['Barri', 'Latitud Barri', 'Longitud Barri']
print(neig_coord_df.shape)
neig_coord_df.head()

(74, 3)


,Barri,Latitud Barri,Longitud Barri
0,el Raval,41.379518,2.168368
1,el Barri Gòtic,41.383395,2.176912
2,la Barceloneta,41.380653,2.189927
3,"Sant Pere, Santa Caterina i la Ribera",0.000000,0.000000
4,el Fort Pienc,41.395925,2.182325


In [33]:
df_barris_cord=pd.merge(df_barris,neig_coord_df,on='Barri',how='outer')
print(df_barris_cord.shape)
df_barris_cord.head()

(74, 5)


,Districte,Codi de Barri,Barri,Latitud Barri,Longitud Barri
0,Ciutat Vella,1,el Raval,41.379518,2.168368
1,Ciutat Vella,2,el Barri Gòtic,41.383395,2.176912
2,Ciutat Vella,3,la Barceloneta,41.380653,2.189927
3,Ciutat Vella,4,"Sant Pere, Santa Caterina i la Ribera",0.000000,0.000000
4,Eixample,5,el Fort Pienc,41.395925,2.182325


In [35]:
df_barris_cord=df_barris_cord.drop(df_barris_cord[df_barris_cord['Latitud Barri']==0].index)
print(df_barris_cord.shape)

(71, 5)


In [79]:
df_comerc=pd.read_csv(r'C:\Users\i2cat\Dades Obertes\Hackovid\2019_censcomercialbcn_detall.csv')
print(df_comerc.columns)
df_comerc.head()

Index(['ID_Bcn_2019', 'ID_Bcn_2016', 'Codi_Principal_Activitat',
       'Nom_Principal_Activitat', 'Codi_Sector_Activitat',
       'Nom_Sector_Activitat', 'Codi_Grup_Activitat', 'Nom_Grup_Activitat',
       'Codi_Activitat_2019', 'Nom_Activitat', 'Codi_Activitat_2016',
       'Nom_Local', 'SN_Oci_Nocturn', 'SN_Coworking', 'SN_Servei_Degustacio',
       'SN_Obert24h', 'SN_Mixtura', 'SN_Carrer', 'SN_Mercat', 'Nom_Mercat',
       'SN_Galeria', 'Nom_Galeria', 'SN_CComercial', 'Nom_CComercial',
       'SN_Eix', 'Nom_Eix', 'X_UTM_ETRS89', 'Y_UTM_ETRS89', 'Latitud',
       'Longitud', 'Direccio_Unica', 'Codi_Via', 'Nom_Via', 'Planta', 'Porta',
       'Num_Policia_Inicial', 'Lletra_Inicial', 'Num_Policia_Final',
       'Lletra_Final', 'Solar', 'Codi_Parcela', 'Codi_Illa', 'Seccio_Censal',
       'Codi_Barri', 'Nom_Barri', 'Codi_Districte', 'Nom_Districte',
       'Referencia_cadastral', 'Data_Revisio'],
      dtype='object')


C:\Users\i2cat\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (35,37) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,ID_Bcn_2019,ID_Bcn_2016,Codi_Principal_Activitat,Nom_Principal_Activitat,Codi_Sector_Activitat,Nom_Sector_Activitat,Codi_Grup_Activitat,Nom_Grup_Activitat,Codi_Activitat_2019,Nom_Activitat,Codi_Activitat_2016,Nom_Local,SN_Oci_Nocturn,SN_Coworking,SN_Servei_Degustacio,SN_Obert24h,SN_Mixtura,SN_Carrer,SN_Mercat,Nom_Mercat,SN_Galeria,Nom_Galeria,SN_CComercial,Nom_CComercial,SN_Eix,Nom_Eix,X_UTM_ETRS89,Y_UTM_ETRS89,Latitud,Longitud,Direccio_Unica,Codi_Via,Nom_Via,Planta,Porta,Num_Policia_Inicial,Lletra_Inicial,Num_Policia_Final,Lletra_Final,Solar,Codi_Parcela,Codi_Illa,Seccio_Censal,Codi_Barri,Nom_Barri,Codi_Districte,Nom_Districte,Referencia_cadastral,Data_Revisio
0,1075454,NaN,1,Actiu,2,Serveis,16,Altres,1600400,Serveis a les empreses i oficines,16004,SORIGUE,1,1,1,1,1,0,1,NaN,1,NaN,1,NaN,1,NaN,427229.272,4577543.637,41.346101,2.130166,"222206, 19-19, LOC 10",222206,MOTORS,LOC,10.0,19,NaN,19,NaN,NaN,NaN,NaN,25.0,12,la Marina del Prat Vermell,3,Sants-Montjuïc,NaN,20190925.0
1,1075453,NaN,1,Actiu,2,Serveis,16,Altres,1600102,Activitats emmagatzematge,1600102,CEJIDOS SIVILA S.A,1,1,1,1,1,0,1,NaN,1,NaN,1,NaN,1,NaN,427178.393,4577526.160,41.345939,2.129560,"222206, 278-282, LOC 10",222206,MOTORS,LOC,10.0,278,NaN,282,NaN,NaN,NaN,NaN,25.0,12,la Marina del Prat Vermell,3,Sants-Montjuïc,NaN,20190925.0
2,1075451,NaN,1,Actiu,2,Serveis,16,Altres,1600400,Serveis a les empreses i oficines,16004,QUALITY ESPRESO,1,1,1,1,1,0,1,NaN,1,NaN,1,NaN,1,NaN,427092.921,4577488.381,41.345591,2.128543,"222206, 1-9, LOC 10",222206,MOTORS,LOC,10.0,1,NaN,9,NaN,NaN,NaN,NaN,25.0,12,la Marina del Prat Vermell,3,Sants-Montjuïc,NaN,20190925.0
3,1075449,NaN,1,Actiu,3,Altres,17,Altres,1700100,Administració,17001,CLD,1,1,1,1,1,0,1,NaN,1,NaN,1,NaN,1,NaN,427265.676,4577561.147,41.346262,2.130599,"222206, 21-21, LOC 10",222206,MOTORS,LOC,10.0,21,NaN,21,NaN,NaN,NaN,NaN,25.0,12,la Marina del Prat Vermell,3,Sants-Montjuïc,NaN,20190925.0
4,1075448,NaN,1,Actiu,2,Serveis,16,Altres,1600101,Activitats de transport,1600101,"CATALANA DEL BUTANO,S.A",1,1,1,1,1,0,1,NaN,1,NaN,1,NaN,1,NaN,427322.177,4577588.560,41.346514,2.131271,"222206, 25-27, LOC 10",222206,MOTORS,LOC,10.0,25,NaN,27,NaN,NaN,NaN,NaN,25.0,12,la Marina del Prat Vermell,3,Sants-Montjuïc,NaN,20190925.0


In [80]:
df_comerc.drop(['ID_Bcn_2016','Codi_Activitat_2016','Codi_Via','Lletra_Inicial', 'Num_Policia_Final',
       'Lletra_Final', 'Solar','Solar', 'Codi_Parcela', 'Codi_Illa', 'Seccio_Censal'],axis=1,inplace=True)

In [81]:
df_comerc.head()

,ID_Bcn_2019,Codi_Principal_Activitat,Nom_Principal_Activitat,Codi_Sector_Activitat,Nom_Sector_Activitat,Codi_Grup_Activitat,Nom_Grup_Activitat,Codi_Activitat_2019,Nom_Activitat,Nom_Local,SN_Oci_Nocturn,SN_Coworking,SN_Servei_Degustacio,SN_Obert24h,SN_Mixtura,SN_Carrer,SN_Mercat,Nom_Mercat,SN_Galeria,Nom_Galeria,SN_CComercial,Nom_CComercial,SN_Eix,Nom_Eix,X_UTM_ETRS89,Y_UTM_ETRS89,Latitud,Longitud,Direccio_Unica,Nom_Via,Planta,Porta,Num_Policia_Inicial,Codi_Barri,Nom_Barri,Codi_Districte,Nom_Districte,Referencia_cadastral,Data_Revisio
0,1075454,1,Actiu,2,Serveis,16,Altres,1600400,Serveis a les empreses i oficines,SORIGUE,1,1,1,1,1,0,1,NaN,1,NaN,1,NaN,1,NaN,427229.272,4577543.637,41.346101,2.130166,"222206, 19-19, LOC 10",MOTORS,LOC,10.0,19,12,la Marina del Prat Vermell,3,Sants-Montjuïc,NaN,20190925.0
1,1075453,1,Actiu,2,Serveis,16,Altres,1600102,Activitats emmagatzematge,CEJIDOS SIVILA S.A,1,1,1,1,1,0,1,NaN,1,NaN,1,NaN,1,NaN,427178.393,4577526.160,41.345939,2.129560,"222206, 278-282, LOC 10",MOTORS,LOC,10.0,278,12,la Marina del Prat Vermell,3,Sants-Montjuïc,NaN,20190925.0
2,1075451,1,Actiu,2,Serveis,16,Altres,1600400,Serveis a les empreses i oficines,QUALITY ESPRESO,1,1,1,1,1,0,1,NaN,1,NaN,1,NaN,1,NaN,427092.921,4577488.381,41.345591,2.128543,"222206, 1-9, LOC 10",MOTORS,LOC,10.0,1,12,la Marina del Prat Vermell,3,Sants-Montjuïc,NaN,20190925.0
3,1075449,1,Actiu,3,Altres,17,Altres,1700100,Administració,CLD,1,1,1,1,1,0,1,NaN,1,NaN,1,NaN,1,NaN,427265.676,4577561.147,41.346262,2.130599,"222206, 21-21, LOC 10",MOTORS,LOC,10.0,21,12,la Marina del Prat Vermell,3,Sants-Montjuïc,NaN,20190925.0
4,1075448,1,Actiu,2,Serveis,16,Altres,1600101,Activitats de transport,"CATALANA DEL BUTANO,S.A",1,1,1,1,1,0,1,NaN,1,NaN,1,NaN,1,NaN,427322.177,4577588.560,41.346514,2.131271,"222206, 25-27, LOC 10",MOTORS,LOC,10.0,25,12,la Marina del Prat Vermell,3,Sants-Montjuïc,NaN,20190925.0


In [82]:
print(df_comerc['Codi_Principal_Activitat'].unique())
print(df_comerc['Nom_Principal_Activitat'].unique())
print(df_comerc['Codi_Sector_Activitat'].unique())
print(df_comerc['Nom_Sector_Activitat'].unique())
print(df_comerc['Nom_Grup_Activitat'].unique())
print(df_comerc['Codi_Activitat_2019'].unique())
print(df_comerc['Nom_Activitat'].unique())


[1 0]
['Actiu' 'Sense activitat Econòmica']
[2 3 1 4 5]
['Serveis' 'Altres' 'Comerç al detall' 'Sense informació'
 'Sense informació - Havia tingut activitat (rètol)' 'En reforma'
 'Locals buits en venda i lloguer' 'Locals buits en lloguer'
 'Locals buits en venda' 'Comerç al detall /Engròs']
['Altres' 'Automoció'
 'Restaurants, bars i hotels (Inclòs hostals, pensions i fondes)'
 'Parament de la llar' 'Quotidià alimentari' 'Quotidià no alimentari'
 'Equipament personal' 'Ensenyament' 'Grup no definit'
 'Reparacions (Electrodomèstics i automòbils)' 'Sanitat i assistència'
 'Finances i assegurances' 'Equipaments culturals i recreatius'
 'Activitats immobiliàries' 'Oci i cultura'
 'Manteniment, neteja i producció']
[1600400 1600102 1700100 1600101 6000000 1400002 3002000 1002000 1000030
 1007000 3001000 1003000 2001000 4003000 1005000 1006000 7004020 1700000
 1000010 2002000 9000000      41 7004010 1300000 1000020 1500000      42
 1100000 1400100 1400001 1111111      50 4002000      30 17

In [83]:
df_comerc=df_comerc.drop(df_comerc[df_comerc['Codi_Principal_Activitat']==0].index)
df_comerc.head()

,ID_Bcn_2019,Codi_Principal_Activitat,Nom_Principal_Activitat,Codi_Sector_Activitat,Nom_Sector_Activitat,Codi_Grup_Activitat,Nom_Grup_Activitat,Codi_Activitat_2019,Nom_Activitat,Nom_Local,SN_Oci_Nocturn,SN_Coworking,SN_Servei_Degustacio,SN_Obert24h,SN_Mixtura,SN_Carrer,SN_Mercat,Nom_Mercat,SN_Galeria,Nom_Galeria,SN_CComercial,Nom_CComercial,SN_Eix,Nom_Eix,X_UTM_ETRS89,Y_UTM_ETRS89,Latitud,Longitud,Direccio_Unica,Nom_Via,Planta,Porta,Num_Policia_Inicial,Codi_Barri,Nom_Barri,Codi_Districte,Nom_Districte,Referencia_cadastral,Data_Revisio
0,1075454,1,Actiu,2,Serveis,16,Altres,1600400,Serveis a les empreses i oficines,SORIGUE,1,1,1,1,1,0,1,NaN,1,NaN,1,NaN,1,NaN,427229.272,4577543.637,41.346101,2.130166,"222206, 19-19, LOC 10",MOTORS,LOC,10.0,19,12,la Marina del Prat Vermell,3,Sants-Montjuïc,NaN,20190925.0
1,1075453,1,Actiu,2,Serveis,16,Altres,1600102,Activitats emmagatzematge,CEJIDOS SIVILA S.A,1,1,1,1,1,0,1,NaN,1,NaN,1,NaN,1,NaN,427178.393,4577526.160,41.345939,2.129560,"222206, 278-282, LOC 10",MOTORS,LOC,10.0,278,12,la Marina del Prat Vermell,3,Sants-Montjuïc,NaN,20190925.0
2,1075451,1,Actiu,2,Serveis,16,Altres,1600400,Serveis a les empreses i oficines,QUALITY ESPRESO,1,1,1,1,1,0,1,NaN,1,NaN,1,NaN,1,NaN,427092.921,4577488.381,41.345591,2.128543,"222206, 1-9, LOC 10",MOTORS,LOC,10.0,1,12,la Marina del Prat Vermell,3,Sants-Montjuïc,NaN,20190925.0
3,1075449,1,Actiu,3,Altres,17,Altres,1700100,Administració,CLD,1,1,1,1,1,0,1,NaN,1,NaN,1,NaN,1,NaN,427265.676,4577561.147,41.346262,2.130599,"222206, 21-21, LOC 10",MOTORS,LOC,10.0,21,12,la Marina del Prat Vermell,3,Sants-Montjuïc,NaN,20190925.0
4,1075448,1,Actiu,2,Serveis,16,Altres,1600101,Activitats de transport,"CATALANA DEL BUTANO,S.A",1,1,1,1,1,0,1,NaN,1,NaN,1,NaN,1,NaN,427322.177,4577588.560,41.346514,2.131271,"222206, 25-27, LOC 10",MOTORS,LOC,10.0,25,12,la Marina del Prat Vermell,3,Sants-Montjuïc,NaN,20190925.0


In [84]:
df_comerc.shape

(61558, 39)

In [85]:
df_comerc=df_comerc.drop(['Codi_Principal_Activitat','Nom_Principal_Activitat','Codi_Sector_Activitat','Nom_Sector_Activitat'],axis=1)
df_comerc.head()

,ID_Bcn_2019,Codi_Grup_Activitat,Nom_Grup_Activitat,Codi_Activitat_2019,Nom_Activitat,Nom_Local,SN_Oci_Nocturn,SN_Coworking,SN_Servei_Degustacio,SN_Obert24h,SN_Mixtura,SN_Carrer,SN_Mercat,Nom_Mercat,SN_Galeria,Nom_Galeria,SN_CComercial,Nom_CComercial,SN_Eix,Nom_Eix,X_UTM_ETRS89,Y_UTM_ETRS89,Latitud,Longitud,Direccio_Unica,Nom_Via,Planta,Porta,Num_Policia_Inicial,Codi_Barri,Nom_Barri,Codi_Districte,Nom_Districte,Referencia_cadastral,Data_Revisio
0,1075454,16,Altres,1600400,Serveis a les empreses i oficines,SORIGUE,1,1,1,1,1,0,1,NaN,1,NaN,1,NaN,1,NaN,427229.272,4577543.637,41.346101,2.130166,"222206, 19-19, LOC 10",MOTORS,LOC,10.0,19,12,la Marina del Prat Vermell,3,Sants-Montjuïc,NaN,20190925.0
1,1075453,16,Altres,1600102,Activitats emmagatzematge,CEJIDOS SIVILA S.A,1,1,1,1,1,0,1,NaN,1,NaN,1,NaN,1,NaN,427178.393,4577526.160,41.345939,2.129560,"222206, 278-282, LOC 10",MOTORS,LOC,10.0,278,12,la Marina del Prat Vermell,3,Sants-Montjuïc,NaN,20190925.0
2,1075451,16,Altres,1600400,Serveis a les empreses i oficines,QUALITY ESPRESO,1,1,1,1,1,0,1,NaN,1,NaN,1,NaN,1,NaN,427092.921,4577488.381,41.345591,2.128543,"222206, 1-9, LOC 10",MOTORS,LOC,10.0,1,12,la Marina del Prat Vermell,3,Sants-Montjuïc,NaN,20190925.0
3,1075449,17,Altres,1700100,Administració,CLD,1,1,1,1,1,0,1,NaN,1,NaN,1,NaN,1,NaN,427265.676,4577561.147,41.346262,2.130599,"222206, 21-21, LOC 10",MOTORS,LOC,10.0,21,12,la Marina del Prat Vermell,3,Sants-Montjuïc,NaN,20190925.0
4,1075448,16,Altres,1600101,Activitats de transport,"CATALANA DEL BUTANO,S.A",1,1,1,1,1,0,1,NaN,1,NaN,1,NaN,1,NaN,427322.177,4577588.560,41.346514,2.131271,"222206, 25-27, LOC 10",MOTORS,LOC,10.0,25,12,la Marina del Prat Vermell,3,Sants-Montjuïc,NaN,20190925.0


In [88]:
print(df_comerc['Referencia_cadastral'].unique())
print(df_comerc['Data_Revisio'].unique())
print(df_comerc['Planta'].unique())
print(df_comerc['Nom_Via'].unique())

[nan '3964201DF3836D' '3867801DF3836F' ... '9134318DF2893C'
 '0521704DF3802B' '1361608DF3816A']
[20190925. 20190912. 20190913. 20191002. 20190903. 20190612. 20190723.
 20190702. 20190619. 20190509. 20190604. 20190527. 20190507. 20190429.
 20190614. 20190713. 20190710. 20190515. 20190705. 20190607. 20190830.
 20190730. 20190703. 20190517. 20190621. 20190722. 20190715. 20190709.
 20190906. 20190731. 20190613. 20190801. 20190513. 20190718. 20190829.
 20190606. 20190620. 20190618. 20190628. 20190528. 20190523. 20190514.
 20190708. 20190807. 20190802. 20190508. 20190529. 20190805. 20190726.
 20190919. 20190729. 20190418. 20190423. 20190408. 20190611. 20190827.
 20190720. 20190711. 20190426. 20190506. 20190717. 20190521. 20190531.
 20190627. 20190617. 20190725. 20190516. 20190701. 20190716. 20190430.
 20190416. 20190603. 20190417. 20190530. 20190823. 20190704. 20190424.
 20190712. 20190522. 20190411. 20190615. 20190808. 20190828. 20190412.
 20190822. 20190625. 20190605. 20190904. 20190719. 2

In [89]:
df_comerc=df_comerc.drop(['Referencia_cadastral','Data_Revisio','Planta'],axis=1)
df_comerc.shape

(61558, 32)